In [1]:
import requests
import schedule
import smtplib
import pickle
import time
import tqdm
import glob
import ssl
import os
import re

import pandas as pd
import numpy  as np

from fake_useragent import UserAgent
from collections    import defaultdict
from statistics     import mean
from torrequest     import TorRequest
from random         import randint
from bs4            import BeautifulSoup as bs

In [2]:
def save(obj, name):
    pickle.dump(obj, open(name + '.pickle', 'wb'))

def load(name):
    return pickle.load(open(name + '.pickle', 'rb'))

In [3]:
def scrape_recent():
    first_page = 'http://www.vdc-sy.info/index.php/en/martyrs/1/c29ydGJ5PWEua2lsbGVkX2RhdGV8c29ydGRpcj1ERVNDfGFwcHJvdmVkPXZpc2libGV8ZXh0cmFkaXNwbGF5PTB8'
    pattern    = re.compile('\/index\.php\/en\/details\/martyrs\/.')

    ua         = UserAgent()
    headers    = {'User-Agent': ua.random}
    tor        = TorRequest(password = 'commonhorse')


    try:
        response = tor.get(first_page, headers=headers)
        content  = bs(response.text, 'html.parser')
        
        links    = {link['href'][30:] for link in content.find_all('a', href = True) if pattern.match(link['href'])} 

    except Exception as e:
        print(e)

    return links

In [4]:
def scrape_details(uid, tor, headers):
    cols = []
    vals = []

    url  = 'http://www.vdc-sy.info/index.php/en/details/martyrs/' + uid
    
    # Headers will provide the UserAgent to use when getting response
    # Makes the request using a TorRequest object passed in
    page = tor.get(url, headers = headers).text
    page = bs(page, 'html.parser')
    
    table = page.find('table', attrs = {'class':'peopleListing'})
    rows  = table.find_all('tr')

    for row in rows:
        data = row.find_all('td')

        # All data without only 2 data values
        # are not data we are looking for
        if len(data) != 2:
            continue

        # data[0] corresponds to the row label/column
        cols.append(data[0].text)
        
        # Values need to appended differently for image rows 
        if data[1].find('img') is not None:
            vals.append(data[1].find('img')['src'])
        else:
            vals.append(data[1].text)

    # Adds the uid to the dataframe
    cols.append('uid')
    vals.append(uid)

    # Creates and saves dataframe
    person = pd.DataFrame([vals], columns = cols, dtype=str)
#     print(person.head())
    save(person, os.path.join('leftovers', uid))

In [5]:
helen___uids_to_scrape = load('helen___uids_to_scrape')
katelyn_uids_to_scrape = load('katelyn_uids_to_scrape')
matthew_uids_to_scrape = load('matthew_uids_to_scrape')
roman___uids_to_scrape = load('roman___uids_to_scrape')

helen___uids_scraped   = load('helen___uids_scraped')
katelyn_uids_scraped   = load('katelyn_uids_scraped')
matthew_uids_scraped   = load('matthew_uids_scraped')
roman___uids_scraped   = load('roman___uids_scraped')

# Helen

In [17]:
ua         = UserAgent()
headers    = {'User-Agent': ua.random}
tor        = TorRequest(password = 'cmps184')
tor.reset_identity()

helen___uids_scraped   = load('helen___uids_scraped')
helen___uids_to_scrape = load('helen___uids_to_scrape')

while len(helen___uids_to_scrape) > 0:
#     time.sleep(1)

    uid = helen___uids_to_scrape.pop()

    try:
        ua         = UserAgent()
        headers    = {'User-Agent': ua.random}
        tor        = TorRequest(password = 'cmps184')
        scrape_details(uid, tor, headers)

    except Exception as e:
        print(e)
        helen___uids_to_scrape.append(uid)

        ua         = UserAgent()
        headers    = {'User-Agent': ua.random}
        tor        = TorRequest(password = 'cmps184')
        tor.reset_identity()

        continue

    print('Left to scrape: ', len(helen___uids_to_scrape))
    helen___uids_scraped.add(uid)

    save(helen___uids_to_scrape, 'helen___uids_to_scrape')
    save(helen___uids_scraped  , 'helen___uids_scraped')

OSError: Process terminated: Failed to bind one of the listener ports.

# Matthew

In [ ]:
ua         = UserAgent()
headers    = {'User-Agent': ua.random}
tor        = TorRequest(password = '')
tor.reset_identity()

matthew_uids_scraped   = load('matthew_uids_scraped')
matthew_uids_to_scrape = load('matthew_uids_to_scrape')

while len(matthew_uids_to_scrape) > 0:
    uid = matthew_uids_to_scrape.pop()

    try:
        scrape_details(uid, tor, headers)

    except Exception as e:
        print(e)
        matthew_uids_to_scrape.append(uid)

        ua         = UserAgent()
        headers    = {'User-Agent': ua.random}
        tor        = TorRequest(password = '')
        tor.reset_identity()

        continue

    print('Left to scrape: ', len(matthew_uids_to_scrape))
    matthew_uids_scraped.add(uid)

    save(matthew_uids_to_scrape, 'matthew_uids_to_scrape')
    save(matthew_uids_scraped  , 'matthew_uids_scraped')

In [ ]:
# Roman

In [18]:
ua         = UserAgent()
headers    = {'User-Agent': ua.random}
tor        = TorRequest(password = 'commonhorse')
tor.reset_identity()

roman___uids_scraped   = load('roman___uids_scraped')
roman___uids_to_scrape = load('roman___uids_to_scrape')

while len(roman___uids_to_scrape) > 0:
    uid = roman___uids_to_scrape.pop()

    try:
        scrape_details(uid, tor, headers)

    except Exception as e:
        print(e)
        roman___uids_to_scrape.append(uid)

        ua         = UserAgent()
        headers    = {'User-Agent': ua.random}
        tor        = TorRequest(password = 'commonhorse')
        tor.reset_identity()

        continue

    print('Left to scrape: ', len(roman___uids_to_scrape))
    matthew_uids_scraped.add(uid)

    save(roman___uids_to_scrape, 'roman___uids_to_scrape')
    save(roman___uids_scraped  , 'roman___uids_scraped')

OSError: Process terminated: Failed to bind one of the listener ports.

In [10]:
len(matthew_uids_scraped)

8499